In [ ]:
import pandas as pd
import numpy as np

# Define the number of observations to generate
num_observations = 10000

# Define the types of shock fronts
shock_front_types = ['normal', 'oblique', 'bow']

# Define possible values or ranges for each feature
mach_number_range = (0.5, 5.0)  # Range of Mach number (subsonic to supersonic)
shock_strength_range = (1.0, 5.0)  # Range of shock strength (low to high)
angles_of_attack = [0, 10, 20, 30, 45, 60, 90]  # Possible angles of attack in degrees
fluids = ['air', 'water']  # Possible fluid media
temperatures_range = (200, 1000)  # Range of initial temperatures in Kelvin
pressures_range = (0.5, 10.0)  # Range of initial pressures in atm

# Initialize lists to hold the generated data
data = []

# Generate the dataset
for _ in range(num_observations):
    # Randomly choose a shock front type (label)
    shock_front_type = np.random.choice(shock_front_types)
    
    # Randomly generate features
    mach_number = np.random.uniform(*mach_number_range)
    shock_strength = np.random.uniform(*shock_strength_range)
    angle_of_attack = np.random.choice(angles_of_attack)
    fluid_medium = np.random.choice(fluids)
    initial_temperature = np.random.uniform(*temperatures_range)
    initial_pressure = np.random.uniform(*pressures_range)
    
    # Append the observation to the data list
    data.append([
        mach_number,
        shock_strength,
        angle_of_attack,
        fluid_medium,
        initial_temperature,
        initial_pressure,
        shock_front_type  # The label for this observation
    ])

# Create a DataFrame from the data list
columns = ['Mach Number', 'Shock Strength', 'Angle of Attack', 'Fluid Medium', 
           'Initial Temperature (K)', 'Initial Pressure (atm)', 'Shock Front Type']
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame as a CSV file
csv_filename = 'shock_front_dataset.csv'
df.to_csv(csv_filename, index=False)

print(f"Dataset saved as '{csv_filename}'")


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('shock_front_dataset.csv')

# Histograms of numerical features
df.hist(column=['Mach Number', 'Shock Strength', 'Angle of Attack', 'Initial Temperature (K)', 'Initial Pressure (atm)'],
        figsize=(10, 8))
plt.suptitle('Histograms of Numerical Features')
plt.show()


# Pair plots to visualize relationships between features
sns.pairplot(df, hue='Shock Front Type', diag_kind='kde')
plt.suptitle('Pair Plots of Features', y=1.02)
plt.show()

# Correlation heatmap of numerical features
corr_matrix = df[['Mach Number', 'Shock Strength', 'Angle of Attack', 'Initial Temperature (K)', 'Initial Pressure (atm)']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap of Numerical Features')
plt.show()



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

# Load the dataset
df = pd.read_csv('shock_front_dataset.csv')

# Separate features and labels
X = df.drop(columns=['Shock Front Type'])
y = df['Shock Front Type']

# Encode categorical features
# Convert categorical features to numerical format
categorical_features = ['Fluid Medium']
X = pd.get_dummies(X, columns=categorical_features)

# Encode labels (Shock Front Type)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert labels to categorical (one-hot encoding)
y_categorical = to_categorical(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Output layer with softmax activation
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks for early stopping and model checkpoint
checkpoint_filepath = 'best_nn_model.keras'
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=10)

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[model_checkpoint, early_stopping])

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")

# Plot training history (accuracy and loss)
plt.figure()
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy/Loss')
plt.legend()
plt.title('Training History')
plt.show()



In [ ]:
# Load the best model
model.load_weights(checkpoint_filepath)

# Inference on new data
def predict_shock_front(features):
    # Ensure features match the input shape of the model
    features = np.array(features).reshape(1, -1)
    
    # Make a prediction
    prediction = model.predict(features)
    
    # Convert prediction to label
    predicted_label = label_encoder.inverse_transform(np.argmax(prediction, axis=1))
    return predicted_label

# Example usage of predict_shock_front function
# Provide new data as input to the function (e.g., as a list of feature values)
new_features = [1.2, 3.4, 45, 500, 1.0, 0, 1]
# Convert features to a numpy array and specify the data type (e.g., float32)
features = np.array(new_features, dtype=np.float32).reshape(1, -1)
predicted_label = predict_shock_front(features)
print("Predicted Label:", predicted_label)
